# Federated Learning

## 01. Example using MNIST

In this notebook we extend the original example of using Flower Federated Learning with scikit-learn (https://flower.dev/docs/quickstart-scikitlearn.html)

* We adapt the online example to make it compatible with a notebook format, using the FL Simulation, over separate client-server Python files.
* We output the individual client accuracy scores to assess their individual and collective performance.
* Minor amendments to enable the example to work within a single Jupyter notebook file.



In [1]:
%%capture
!pip install flwr[simulation] torch torchvision matplotlib

In [2]:
from collections import OrderedDict
from typing import List, Tuple

import flwr as fl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import warnings


from typing import Dict

from sklearn.metrics import log_loss

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier


print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.0.0
numpy 1.22.4
torch 1.12.1+cpu
torchvision 0.13.1+cpu
Training on cpu


In [3]:
from typing import Tuple, Union, List
import numpy as np
from sklearn.linear_model import LogisticRegression
import openml

XY = Tuple[np.ndarray, np.ndarray]
Dataset = Tuple[XY, XY]
LogRegParams = Union[XY, Tuple[np.ndarray]]
XYList = List[XY]

def get_model_parameters(model: LogisticRegression) -> LogRegParams:
    """Returns the paramters of a sklearn LogisticRegression model."""
    if model.fit_intercept:
        params = [
            model.coef_,
            model.intercept_,
        ]
    else:
        params = [
            model.coef_,
        ]
    return params


def set_model_params(
    model: LogisticRegression, params: LogRegParams
) -> LogisticRegression:
    """Sets the parameters of a sklean LogisticRegression model."""
    model.coef_ = params[0]
    if model.fit_intercept:
        model.intercept_ = params[1]
    return model


def set_initial_params(model: LogisticRegression):
    """Sets initial parameters as zeros Required since model params are
    uninitialized until model.fit is called.
    But server asks for initial parameters from clients at launch. Refer
    to sklearn.linear_model.LogisticRegression documentation for more
    information.
    """
    n_classes = 10 # CICIDS has 15 possible classes #10  # MNIST has 10 classes
    n_features = 784 # CICIDS has 78 posible features #784  # Number of features in dataset
    model.classes_ = np.array([i for i in range(10)])

    model.coef_ = np.zeros((n_classes, n_features))
    if model.fit_intercept:
        model.intercept_ = np.zeros((n_classes,))


def load_mnist() -> Dataset:
    """Loads the MNIST dataset using OpenML.
    OpenML dataset link: https://www.openml.org/d/554
    """
    mnist_openml = openml.datasets.get_dataset(554)
    Xy, _, _, _ = mnist_openml.get_data(dataset_format="array")
    X = Xy[:, :-1]  # the last column contains labels
    y = Xy[:, -1]
    # First 60000 samples consist of the train set
    x_train, y_train = X[:60000], y[:60000]
    x_test, y_test = X[60000:], y[60000:]
    return (x_train, y_train), (x_test, y_test)


def shuffle(X: np.ndarray, y: np.ndarray) -> XY:
    """Shuffle X and y."""
    rng = np.random.default_rng()
    idx = rng.permutation(len(X))
    return X[idx], y[idx]


def partition(X: np.ndarray, y: np.ndarray, num_partitions: int) -> XYList:
    """Split X and y into a number of partitions."""
    return list(
        zip(np.array_split(X, num_partitions), np.array_split(y, num_partitions))
    )


In [4]:
NUM_OF_CLIENTS = 10
NUM_OF_ROUNDS = 20

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, c_model, c_X_train, c_y_train, c_X_test, c_y_test):
        self.cid = cid
        self.c_model = c_model
        self.c_X_train = c_X_train
        self.c_y_train = c_y_train
        self.c_X_test = c_X_test
        self.c_y_test = c_y_test

    def get_parameters(self, config):  # type: ignore
        #print (f"Get Parameters for Client {self.cid}")
        return get_model_parameters(self.c_model)

    def fit(self, parameters, config):  # type: ignore
        #print (f"Fit for Client {self.cid}")
        set_model_params(self.c_model, parameters)
        # Ignore convergence failure due to low local epochs
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.c_model.fit(self.c_X_train, self.c_y_train)
        print(f"Training finished for round {config['server_round']} for Client {self.cid}")
        return get_model_parameters(self.c_model), len(self.c_X_train), {}

    def evaluate(self, parameters, config):  # type: ignore
        #print (f"Evaluate for Client {self.cid}")
        set_model_params(self.c_model, parameters)
        #c_loss = log_loss(self.c_y_test, self.c_model.predict_proba(self.c_X_test))
        #c_accuracy = self.c_model.score(self.c_X_test, self.c_y_test)
        c_loss = log_loss(self.c_y_train, self.c_model.predict_proba(self.c_X_train))
        c_accuracy = self.c_model.score(self.c_X_train, self.c_y_train)
        print (f"Evaluate for Client {self.cid} - Accuracy: {c_accuracy}, Loss: {c_loss}")
        return c_loss, len(self.c_X_train), {"accuracy": c_accuracy}


In [6]:



def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Split train set into 10 partitions and randomly use one for training.
    # Load MNIST dataset from https://www.openml.org/d/554
    (X_train, y_train), (X_test, y_test) = load_mnist()
    partition_id = np.random.choice(10)
    (X_train, y_train) = partition(X_train, y_train, 10)[partition_id]

    # Create LogisticRegression Model
    client_model = LogisticRegression(
        penalty="l2",
        max_iter=1,  # local epoch
        warm_start=True,  # prevent refreshing weights when fitting
    )

    # Setting initial parameters, akin to model.compile for keras models
    set_initial_params(client_model)

    # Create a single Flower client representing a single organization
    return FlowerClient(cid, client_model, X_train, y_train, X_test, y_test)

In [7]:
def fit_round(server_round: int) -> Dict:
    """Send round number to client."""
    return {"server_round": server_round}


def get_evaluate_fn(model: LogisticRegression):
    """Return an evaluation function for server-side evaluation."""

    # Load test data here to avoid the overhead of doing it in `evaluate` itself
    _, (s_X_test, s_y_test) = load_mnist()
    
    # The `evaluate` function will be called after every round
    def evaluate(server_round, parameters: fl.common.NDArrays, config):
        # Update model with the latest parameters
        set_model_params(model, parameters)
        loss = log_loss(s_y_test, model.predict_proba(s_X_test))
        accuracy = model.score(s_X_test, s_y_test)
        return loss, {"accuracy": accuracy}

    return evaluate

server_model = LogisticRegression()
set_initial_params(server_model)

strategy = fl.server.strategy.FedAvg(
    min_available_clients=2,
    evaluate_fn=get_evaluate_fn(server_model),
    on_fit_config_fn=fit_round,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    
)

INFO flower 2022-10-26 21:35:17,222 | app.py:142 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)
INFO flower 2022-10-26 21:35:23,923 | app.py:176 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 10789785600.0, 'CPU': 8.0, 'object_store_memory': 5394892800.0}
INFO flower 2022-10-26 21:35:23,926 | server.py:86 | Initializing global parameters
INFO flower 2022-10-26 21:35:23,927 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-10-26 21:35:27,983 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-10-26 21:35:27,984 | server.py:88 | Evaluating initial parameters
INFO flower 2022-10-26 21:35:28,150 | server.py:91 | initial parameters (loss, other metrics): 2.3025850929940455, {'accuracy': 0.098}
INFO flower 2022-10-26 21:35:28,152 | server.py:101 | FL starting
DEBUG flower 2022-10-26 21:35:28,154 | server.py:215 | fit_round 1: strategy sampled 10 clie

(launch_and_fit pid=23756) Training finished for round 1 for Client 4
(launch_and_fit pid=23756) Training finished for round 1 for Client 9
(launch_and_fit pid=23756) Training finished for round 1 for Client 6
(launch_and_fit pid=22316) Training finished for round 1 for Client 7
(launch_and_fit pid=23748) Training finished for round 1 for Client 0
(launch_and_fit pid=23148) Training finished for round 1 for Client 1
(launch_and_fit pid=17404) Training finished for round 1 for Client 3
(launch_and_fit pid=19232) Training finished for round 1 for Client 5


DEBUG flower 2022-10-26 21:35:35,477 | server.py:229 | fit_round 1 received 10 results and 0 failures
WARNING flower 2022-10-26 21:35:35,493 | fedavg.py:243 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=5228) Training finished for round 1 for Client 2
(launch_and_fit pid=3608) Training finished for round 1 for Client 8


INFO flower 2022-10-26 21:35:35,982 | server.py:116 | fit progress: (1, 1.3277909087574342, {'accuracy': 0.68}, 7.828432600000003)
DEBUG flower 2022-10-26 21:35:35,984 | server.py:165 | evaluate_round 1: strategy sampled 10 clients (out of 10)


...

(launch_and_evaluate pid=22316) Evaluate for Client 4 - Accuracy: 0.8803333333333333, Loss: 0.4210924474380631
(launch_and_evaluate pid=23748) Evaluate for Client 7 - Accuracy: 0.8928333333333334, Loss: 0.3920644841508133
(launch_and_evaluate pid=23756) Evaluate for Client 3 - Accuracy: 0.891, Loss: 0.38920021735238924
(launch_and_evaluate pid=5228) Evaluate for Client 2 - Accuracy: 0.8928333333333334, Loss: 0.3920644841508133
(launch_and_evaluate pid=23148) Evaluate for Client 8 - Accuracy: 0.8798333333333334, Loss: 0.41933273651806907
(launch_and_evaluate pid=19232) Evaluate for Client 0 - Accuracy: 0.9046666666666666, Loss: 0.3346912832126185
(launch_and_evaluate pid=3608) Evaluate for Client 5 - Accuracy: 0.8855, Loss: 0.40693791320901535
(launch_and_evaluate pid=17404) Evaluate for Client 6 - Accuracy: 0.8803333333333333, Loss: 0.4210924474380631


DEBUG flower 2022-10-26 21:36:48,735 | server.py:179 | evaluate_round 20 received 10 results and 0 failures
INFO flower 2022-10-26 21:36:48,738 | server.py:144 | FL finished in 80.5837344
INFO flower 2022-10-26 21:36:48,740 | app.py:180 | app_fit: losses_distributed [(1, 1.3362465672042048), (2, 0.7356410930330116), (3, 0.5563196543571093), (4, 0.5468685054798318), (5, 0.5387834402091146), (6, 0.5220944687566411), (7, 0.5244525070308981), (8, 0.4976500561301475), (9, 0.48839422310274894), (10, 0.4786086862842617), (11, 0.4603521362783348), (12, 0.4522970204750947), (13, 0.45261459992654346), (14, 0.4476913481426404), (15, 0.44494037105032586), (16, 0.43583495162955704), (17, 0.43136925864078124), (18, 0.40701718036920037), (19, 0.4156305131922239), (20, 0.3998154124023924)]
INFO flower 2022-10-26 21:36:48,742 | app.py:181 | app_fit: metrics_distributed {}
INFO flower 2022-10-26 21:36:48,743 | app.py:182 | app_fit: losses_centralized [(0, 2.3025850929940455), (1, 1.3277909087574342), (2

(launch_and_evaluate pid=22316) Evaluate for Client 1 - Accuracy: 0.8748333333333334, Loss: 0.42961362640326534


History (loss, distributed):
	round 1: 1.3362465672042048
	round 2: 0.7356410930330116
	round 3: 0.5563196543571093
	round 4: 0.5468685054798318
	round 5: 0.5387834402091146
	round 6: 0.5220944687566411
	round 7: 0.5244525070308981
	round 8: 0.4976500561301475
	round 9: 0.48839422310274894
	round 10: 0.4786086862842617
	round 11: 0.4603521362783348
	round 12: 0.4522970204750947
	round 13: 0.45261459992654346
	round 14: 0.4476913481426404
	round 15: 0.44494037105032586
	round 16: 0.43583495162955704
	round 17: 0.43136925864078124
	round 18: 0.40701718036920037
	round 19: 0.4156305131922239
	round 20: 0.3998154124023924
History (loss, centralized):
	round 0: 2.3025850929940455
	round 1: 1.3277909087574342
	round 2: 0.7227150311788029
	round 3: 0.5341110766653655
	round 4: 0.5095080068756908
	round 5: 0.49801024524231263
	round 6: 0.4871907655614814
	round 7: 0.4829463515371517
	round 8: 0.4703486550784219
	round 9: 0.4567570937565834
	round 10: 0.4440053076505489
	round 11: 0.44299782097

(launch_and_evaluate pid=23748) Evaluate for Client 9 - Accuracy: 0.8928333333333334, Loss: 0.3920644841508133
